# Mini Dollar Strategy - Data Analysis and Technical Indicators

This notebook demonstrates:
1. Loading market data
2. Exploratory data analysis
3. Technical indicators calculation
4. Pattern visualization

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from src.data.loaders.market_data import MarketDataLoader
from src.analysis.technical.strategy import TechnicalStrategy

# Configure plotting
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]
sns.set_palette('Set2')

## 1. Load and Prepare Market Data

In [ ]:
# Initialize components
loader = MarketDataLoader()

# Load last 6 months of data
end_date = datetime.now()
start_date = end_date - timedelta(days=180)

data = loader.load_data(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'),
    timeframe='5T'
)

print(f"Loaded {len(data)} candles")
data.head()

### Data Quality Analysis

In [ ]:
def analyze_data_quality(df):
    """Analyze data quality and print report."""
    print("Data Quality Report:\n")
    
    # Basic information
    print("1. Basic Information:")
    print(f"Total records: {len(df)}")
    print(f"Date range: {df.index.min()} to {df.index.max()}")
    print(f"Columns: {', '.join(df.columns)}\n")
    
    # Missing values
    print("2. Missing Values:")
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("No missing values found\n")
    
    # Data statistics
    print("3. Data Statistics:")
    print(df.describe())

# Run analysis
analyze_data_quality(data)

## 2. Calculate Technical Indicators

In [ ]:
# Initialize strategy
strategy = TechnicalStrategy()

# Calculate indicators
analysis = strategy.calculate_indicators(data)

# Show available indicators
print("Available indicators:")
indicators = [col for col in analysis.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
print(indicators)
analysis.head()

## 3. Visualize Price Action and Indicators

In [ ]:
def plot_price_and_indicators(df, window=500):
    """Plot comprehensive price and indicator analysis."""
    recent = df.tail(window)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(15, 12))
    
    # 1. Price and Moving Averages
    ax1 = plt.subplot(3, 1, 1)
    ax1.plot(recent.index, recent['close'], label='Price', alpha=0.8)
    ax1.plot(recent.index, recent['ma_fast'], label='Fast MA', alpha=0.7)
    ax1.plot(recent.index, recent['ma_slow'], label='Slow MA', alpha=0.7)
    ax1.plot(recent.index, recent['bb_upper'], 'k--', label='BB Upper', alpha=0.5)
    ax1.plot(recent.index, recent['bb_lower'], 'k--', label='BB Lower', alpha=0.5)
    ax1.set_title('Price Action with Indicators')
    ax1.legend()
    ax1.grid(True)
    
    # 2. RSI
    ax2 = plt.subplot(3, 1, 2)
    ax2.plot(recent.index, recent['rsi'])
    ax2.axhline(y=70, color='r', linestyle='--')
    ax2.axhline(y=30, color='g', linestyle='--')
    ax2.set_title('RSI')
    ax2.grid(True)
    
    # 3. MACD
    ax3 = plt.subplot(3, 1, 3)
    ax3.plot(recent.index, recent['macd'], label='MACD')
    ax3.plot(recent.index, recent['macd_signal'], label='Signal')
    ax3.bar(recent.index, recent['macd_hist'], alpha=0.3, label='Histogram')
    ax3.set_title('MACD')
    ax3.legend()
    ax3.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot analysis
plot_price_and_indicators(analysis)

## 4. Analyze Potential Trading Signals

In [ ]:
# Generate trading signals
signals = strategy.generate_signals(analysis)

def analyze_trading_signals(df):
    """Analyze generated trading signals."""
    # Count signals
    signal_counts = df['final_signal'].value_counts()
    print("Signal Distribution:")
    print(f"Buy signals (1): {signal_counts.get(1, 0)}")
    print(f"Sell signals (-1): {signal_counts.get(-1, 0)}")
    print(f"Neutral (0): {signal_counts.get(0, 0)}\n")
    
    # Signal transitions
    transitions = df['final_signal'].diff().value_counts()
    print("Signal Transitions:")
    print(transitions)

# Analyze signals
analyze_trading_signals(signals)

# Plot recent signals
recent = signals.tail(100)
plt.figure(figsize=(15, 6))
plt.plot(recent.index, recent['close'], label='Price')
plt.scatter(recent[recent['final_signal'] == 1].index, 
            recent[recent['final_signal'] == 1]['close'],
            marker='^', c='g', s=100, label='Buy')
plt.scatter(recent[recent['final_signal'] == -1].index,
            recent[recent['final_signal'] == -1]['close'],
            marker='v', c='r', s=100, label='Sell')
plt.title('Recent Trading Signals')
plt.legend()
plt.grid(True)
plt.show()